### Plan of Attack:

1. Get list of new songs after a specific date
2. Get data needed for those songs
3. Create parquet table
4. Save to S3

In [ ]:
# import spotipy
# import os
# %run './credentials/settings.py'
# from spotipy.oauth2 import SpotifyOAuth
# scope = "user-library-read"

# def user_auth(scope=scope):
#     SPOTIPY_CLIENT_ID = os.getenv("SPOTIPY_CLIENT_ID")
#     SPOTIPY_CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET")
#     SPOTIPY_REDIRECT_URI = os.getenv('SPOTIPY_REDIRECT_URI_READ')
#     print()
#     sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
#     return sp
# sp = user_auth()

In [4]:
!pip install spotipy

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
aiobotocore 1.3.3 requires botocore<1.20.107,>=1.20.106, but you have botocore 1.21.2 which is incompatible.
You shoul

In [7]:
os.getcwd()

'/root/spotify-recommender/dev_notebooks/max'

In [16]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import os
%run '../../credentials/settings.py'
from collections import defaultdict

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"),
                                                           client_secret=os.getenv("SPOTIPY_CLIENT_SECRET")))


def find_song(name, year):
  
    """
    This function returns a dataframe with data for a song given the name and release year.
    The function uses Spotipy to fetch audio features and metadata for the specified song.
    
    """
    
    song_data = defaultdict()
    results = sp.search(q= 'track: {} year: {}'.format(name,
                                                       year), limit=1)
    if results['tracks']['items'] == []:
        return None
    
    results = results['tracks']['items'][0]

    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]
    
    song_data['name'] = [name]
    song_data['year'] = [year]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]
    
    for key, value in audio_features.items():
        song_data[key] = value
    
    return pd.DataFrame(song_data)
    